# Practice 1: Multitask learning for facial recognition

Paper: Multi-task learning for smile detection, emotion recognition and gender classification (https://www.researchgate.net/publication/321736118_Multi-task_learning_for_smile_detection_emotion_recognition_and_gender_classification)

Trong bài thực hành này, chúng ta sẽ lập trình mô hình Multi-task learning để giải quyết đồng thời 3 bài toán: Phát hiện nụ cười, nhận diện cảm xúc và phân loại giới tính. Bạn sẽ lập trình các đoạn mã nguồn sử dụng các thư viện sau:

- TensorFlow 1.x
- OpenCV, Numpy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/rubik_project

/content/drive/My Drive/rubik_project


## Phần 1: Dataset and Data provider

Trong phần này ta sẽ tìm hiểu các bộ dữ liệu sử dụng và viết các hàm generate dữ liệu phục vụ cho quá trình huấn luyện

### Dataset
Ta sử dụng 3 dataset khác nhau cho 3 bài toán:
- Smile dataset: GENKI-4K
- Emotion dataset: FER-2013
- Gender dataset: WIKI

Để tiện thực hành, chúng ta sẽ sử dụng dữ liệu đã được nén sẵn, việc đọc dữ liệu sử dụng các đoạn code có sẵn trong file data_utils.py. Cụ thể như sau:
- Bộ dữ liệu GENKI-4K: Thư mục data/smile_data gồm 2 file: train.npy chứa 3000 ảnh trong tập train và file test.npy chứa 1000 ảnh trong tập test.
- Bộ dữ liệu FER-2013: Thư mục data/emotion_data2: Gồm tập dữ liệu train gồm hơn 28K ảnh, 2 tập test: public và private, mỗi tập gồm gần 3600 ảnh.
- Bộ dữ liệu WIKI: Thư mục data/gender_data: Gồm tập dữ liệu train gồm 30K ảnh, tập test gồm 3K ảnh


Các ảnh trong bộ dữ liệu đều được chuẩn hóa về kích thước 48x48 ở dạng grayscale. Quan sát hàm read_data trong file data_provider.py, hàm đọc dữ liệu vào các biến:
- Biến smile_train: Là một python list có độ dài 3000, mỗi phần tử của list là một tuple bao gồm ma trận mô tả ảnh và một số nguyên mô tả nhãn dữ liệu tương ứng (0 = không cười, 1 = cười)
- Biến smile_test: Tương tự như smile_train nhưng độ dài list là 1000
- Biến gender_train và gender_test: Tương tự như smile_train và smile_test, nhãn dữ liệu 0 = nữ, 1 = nam
- Biến emotion_train, emotion_public_test, emotion_private_test: Tương tự như trên tuy nhiên ảnh đã được normalize (mỗi pixel p được normalize theo công thức p = (p - 128) / 255), nhãn dữ liệu đã được chuẩn hóa dưới dạng one-hot vector.


### Data provider
Hãy hoàn thành các hàm trong data_provider.py theo gợi ý:
- Hoàn thành hàm get_one_hot_vector() và convert_data_format(): Quan sát cách normalize và chuẩn hóa dữ liệu, hoàn thành việc load data vào biến all_data phụ thuộc vào tham số data_type, nếu data_type='train' thì all_data chứa smile train, emotion train và gender train, nếu data_type='public_test' thì all_data chứa smile test, emotion public test và gender test, nếu data_type='private test' thì all_data chứa smile test, emotion private test và gender test. all_data là một list chứa các tuple (img, label, index), trong đó img được normalize giống theo cách normalize emotion data, label dưới dạng one_hot vector và index mô tả nhãn bài toán của dữ liệu (nhãn 0 ứng với smile data, 1 ứng với emotion data và 2 ứng với gender data). Chú ý rằng data emotion đã được chuẩn hóa sẵn. Check code của bạn với việc chạy thông cell bên dưới:

In [0]:
import numpy as np
import data_provider

dataset = data_provider.Dataset(data_type='public_test', batch_size=64)

assert len(dataset.all_data) == 8707

for i in range(len(dataset.all_data)):
    label = np.squeeze(np.array(dataset.all_data[i][1]))
    index = dataset.all_data[i][2]

    assert np.sum(label) == 1
    
    assert index == 1.0 or index == 2.0 or index == 3.0
    assert label.shape == (7,)
    


Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------


- Hoàn thành hàm gen() để sinh dữ liệu sử dụng hàm yield trong Python. Lưu ý shuffle dữ liệu khi hoàn thành 1 epoch. Gợi ý: Hãy quan sát kết quả của đoạn code sau:

In [0]:
a = list(np.arange(0, 10))
print(a)
def generator(tensor, batch_size=2):
    batch_data = []
    np.random.shuffle(a)
    for i in range(len(tensor)):
        batch_data.append(tensor[i])
        if len(batch_data) == batch_size:
            yield batch_data
            batch_data = []

num_epochs = 2
for epoch in range(num_epochs):
    for batch_data in generator(a, batch_size=2):
        print('Epoch ', epoch, ' batch_data = ', batch_data)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Epoch  0  batch_data =  [2, 9]
Epoch  0  batch_data =  [0, 6]
Epoch  0  batch_data =  [4, 3]
Epoch  0  batch_data =  [7, 1]
Epoch  0  batch_data =  [5, 8]
Epoch  1  batch_data =  [9, 6]
Epoch  1  batch_data =  [5, 2]
Epoch  1  batch_data =  [0, 7]
Epoch  1  batch_data =  [4, 3]
Epoch  1  batch_data =  [8, 1]


Hãy hoàn thành hàm gen() và check với đoạn code trong cell tiếp theo:

In [0]:
dataset = data_provider.Dataset(data_type='public_test', batch_size=1000)
cnt = 0
for batch_images, batch_labels, batch_indexes in dataset.gen():
    cnt += 1
    n = len(batch_images)
    m = len(batch_labels)
    k = len(batch_indexes)
    print(cnt, n, m, k)
    assert n == m == k
    if cnt % 9 == 0:
        assert n == 707
    else:
        assert n == 1000

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------
1 1000 1000 1000
2 1000 1000 1000
3 1000 1000 1000
4 1000 1000 1000
5 1000 1000 1000
6 1000 1000 1000
7 1000 1000 1000
8 1000 1000 1000
9 707 707 707


## Phần 2: Định nghĩa mô hình và huấn luyện 

Trong phần này bạn sẽ lập trình các hàm trong file model.py, bao gồm: _define_input, _build_model, _define_loss, train.

### Định nghĩa các tensor đầu vào 
Hãy hoàn thành hàm _define_input() khởi tạo 5 tensor sau:
- input_images: Ma trận 4 chiều lần lượt là batch_size, chiều rộng ảnh, chiều cao ảnh, số lượng kênh màu
- input_labels: tensor chứa các one-hot vector
- input_indexes: tensor chứa chỉ số miêu tả bài toán ứng với dữ liệu
- phase_train: boolean
- keep_prob: float

### Xây dựng mô hình
Sử dụng các hàm đã lập trình sẵn để xây dựng lại mô hình Multi-task BKNet như trong paper. Hoàn thành hàm _build_model() sử dụng những layer đã được lập trình sẵn

### Lập trình hàm loss
Cả ba bài toán đều sử dụng hàm mất mát cross entropy, vấn đề hiện tại là dữ liệu từ một task bất kỳ sẽ đi ra ở cả 3 đầu của mô hình, tuy nhiên khi tính hàm mất mát, ta chỉ lọc ra những dữ liệu của bài toán tương ứng. Sử dụng thông tin về chỉ số task (index) mà ta đã khởi tạo kèm dữ liệu, hoàn thành hàm _define_loss()

### Huấn luyện mô hình
Sử dụng data_provider và các tensor đã định nghĩa viết hàm train() để thực hiện huấn luyện mô hình.

In [3]:
import model 
import tensorflow as tf
session = tf.Session()
multitask_model = model.Multitask_BKNet(session)
multitask_model.train()





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(?, 24, 24, 32)
(?, 12, 12, 64)
(?, 6, 6, 128)
(?, 3, 3, 256)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------

Instructions for updating:
Please use tf.global_variables instead.
Created model with fresh parameters.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:  0
smile_loss: 0.37, emotion_los

## Bài tập về nhà

Sử dụng những đoạn code bạn đã viết trong giờ học thực hành, viết chương trình load lại model đã huấn luyện và đánh giá độ chính xác trên các tập test của 3 bộ dữ liệu.